# TrueSkill model for 2 players

In [1]:
# Torch and Pyro
import torch._numpy as np
import pyro
import pyro.distributions as dist
from pyro.infer.autoguide import AutoNormal
from pyro.infer import Predictive

# Plotting
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_white"

# Project specific utilities
from utils import *

First we define our 2 player model in Pyro. 

In [2]:
def TwoPlayer(game_info, obs=None):
    """
    Inputs: 
        game_info: A dictionary containing relevant information about the games played.
        obs: A 1D tensor of observed data. The length should be equal to the number of games played.
             Represents the outcome of a game in the eyes of coach 1, i.e. coach 1 wins: obs = 1, coach 2 wins: obs = -1.

    Output: A 1D tensor of the same length as the input tensor. Represents a sample from difference in performance between the two coaches.
    """
    
    # define hyperparameters
    hyper_sigma = 1/8
    
    coach1_mu_skill = 0
    coach1_sigma_skill = 1

    coach2_mu_skill = 0
    coach2_sigma_skill = 1

    # initialize the skill of the coaches
    coach1_skill = pyro.sample("coach1_skill", dist.Normal(coach1_mu_skill, coach1_sigma_skill))
    coach2_skill = pyro.sample("coach2_skill", dist.Normal(coach2_mu_skill, coach2_sigma_skill))
    
    # define plate for the number of games played
    with pyro.plate('matches', obs.shape[0]):   

        # sample the performance of each coach
        coach1_perf = pyro.sample('coach1_perf', dist.Normal(coach1_skill, hyper_sigma))
        coach2_perf = pyro.sample('coach2_perf', dist.Normal(coach2_skill, hyper_sigma))

        # calculate the difference in performance
        perf_diff = coach1_perf - coach2_perf
    
        # sample the outcome of the game
        y = pyro.sample("y_coach1_win", dist.Normal(perf_diff, hyper_sigma), obs=obs)

    return y

First we want to check if the model behaves as expected, so we test it out with some dummy data. First we make dummy data where player 1 always wins. 

In [3]:
num_games = 100
game_info = {}
obs = torch.ones(num_games)

guide = AutoNormal(TwoPlayer)
losses = run_inference(TwoPlayer, guide, game_info, obs)

Loss = -23.880997: 100%|██████████| 2000/2000 [00:16<00:00, 118.21it/s]


Generate samples from the distribution of each players skill.

In [4]:
predictive = Predictive(TwoPlayer, guide=guide, num_samples=2000, return_sites=("coach1_skill", "coach2_skill"))
samples = predictive(game_info, obs)

We plot the samples from the posterior distribution of each players skill.

In [5]:
# Plot the results
fig = go.Figure()
fig.add_trace(go.Histogram(x=samples["coach1_skill"].detach().squeeze(), histnorm='probability density', name="Coach 1"))
fig.add_trace(go.Histogram(x=samples["coach2_skill"].detach().squeeze(), histnorm='probability density', name="Coach 2"))
fig.update_layout(barmode='overlay', xaxis_title="Skill", yaxis_title="Density", title="Coach skill", width=600)
fig.show()

It is clear to see after winning 100 games in a row, then the model has updated the players skills accordingly. Player 1 now has a higher skill than player 2.

For the second dummy data set we set each match to be a draw.

In [6]:
obs = torch.zeros(num_games)
guide = AutoNormal(TwoPlayer)
losses = run_inference(TwoPlayer, guide, game_info, obs)

Loss = -20.070332: 100%|██████████| 2000/2000 [00:16<00:00, 123.99it/s]


In [7]:
predictive = Predictive(TwoPlayer, guide=guide, num_samples=2000, return_sites=("coach1_skill", "coach2_skill"))
samples = predictive(game_info, obs)

In [8]:
# Plot the results
fig = go.Figure()
fig.add_trace(go.Histogram(x=samples["coach1_skill"].detach().squeeze(), histnorm='probability density', name="Coach 1", opacity=0.75))
fig.add_trace(go.Histogram(x=samples["coach2_skill"].detach().squeeze(), histnorm='probability density', name="Coach 2", opacity=0.75))
fig.update_layout(barmode='overlay', xaxis_title="Skill", yaxis_title="Density", title="Coach skill", width=600)
fig.show()

One important thing to note from these posteriors, is that even though the players should be completely equal as 100 games have resulted in a draw. However, the distributions indicate that coach 1 is slightly better than coach 2. Since the output of the model is continuous, then you have to be careful when interpreting the results. This will be important later when discussing the draw margin mentioned in the report.